# Building Abstractions with Procedures

## Exercise 1.1
Below is a sequence of expressions. What is the result printed by the interpreter in response to each expression? Assume that the sequence is to be evaluated in the order in which it is presented
```racket
10
(+ 5 3 4)
(- 9 1)
(/ 6 2)
(+ (* 2 4) (- 4 6))
(define a 3)
(define b (+ a 1))
(+ a b (* a b))
(= a b)
(if (and (> b a) (< b (* a b)))
    b
    a)
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))
(+ 2 (if (> b a) b a))
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))
```

In [1]:
10

10

In [2]:
(+ 5 3 4)

12

In [3]:
(- 9 1)

8

In [4]:
(/ 6 2)

3

In [5]:
(+ (* 2 4) (- 4 6))

6

In [6]:
(define a 3)
(define b (+ a 1))
(+ a b (* a b))

19

In [7]:
(= a b)

#f

In [8]:
(if (and (> b a) (< b (* a b)))
    b
    a)

4

In [9]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))

16

In [10]:
(+ 2 (if (> b a) b a))

6

In [11]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))

16

## Exercise 1.2
Translate the following expression into prefix form
$$
\frac{5+4+(2-(3-(6+\frac{4}{3})))}{3(6-2)(2-7)}
$$

In [12]:
(/ (+ 5 4 (- 2
             (- 3
                (+ 6 (/ 4 3)))))
   (* 3 (- 6 2) (- 2 7)))

-23/90

## Exercise 1.3
Define a procedure that takes three numbers as arguments and returns the sum of the squares of the two larger numbers.

In [13]:
(define (square x)
  (* x x))
(define (sum_of_squares a b)
  (+ (square a) (square b)))
(define (sum_square_of_two_larger x y z)
  (cond ((and (> x z) (> y z))
        (sum_of_squares x y))
        ((> x y)
         (sum_of_squares x z))
        (else (sum_of_squares y z))))

## Exercise 1.4
Observe that our model of evaluation allows for combinations whose operators are compound expressions. Use this observation to describe the behavior of the following procedure:
```racket
(define (a-plus-abs-b a b)
  ((if (> b 0) + -) a b))
```

When we evalute `(a-plus-abs-b a b)`, operator part `(if (> b 0) + -)` will be evaluted first. If `b` is positive, then `(+ a b)` will be evaluated. If `b` is negative, then `(- a b)` will be evaluated.

## Exercise 1.5
Ben Bitdiddle has invented a test to determine whether the interpreter he is faced with is using applicative-order evaluation or normal-order evaluation. He defines the following two procedures:
```racket
(define (p) (p))

(define (test x y)
  (if (= x 0)
      0
      y))
```
Then he evaluates the expression
```racket
(test 0 (p))
```
What behavior will Ben observe with an interpreter that uses applicative-order evaluation? What behavior will he observe with an interpreter that uses normal-order evaluation? Explain your answer. (Assume that the evaluation rule for the special form `if` is the same whether the interpreter is using normal or applicative order: The predicate expression is evaluated first, and the result determines whether to evaluate the consequent or the alternative expression.)

Applicative-order:</br>
The operator with `if` will be evalute first, then we get 0 directly.</br>
Normal-order:</br>
To substitute `(p)`, we get another `(p)` in `(test 0 (p))` , it's a infinite loop.</br>

## Exercise 1.6
Alyssa P. Hacker doesn't see why `if` needs to be provided as a special form. ``Why can't I just define it as an ordinary procedure in terms of `cond`?'' she asks. Alyssa's friend Eva Lu Ator claims this can indeed be done, and she defines a new version of `if`:

```racket
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))
```
Eva demonstrates the program for Alyssa:

```racket
(new-if (= 2 3) 0 5)
5
```

```racket
(new-if (= 1 1) 0 5)
0
```
Delighted, Alyssa uses `new-if` to rewrite the square-root program:

```racket
(define (sqrt-iter guess x)
  (new-if (good-enough? guess x)
          guess
          (sqrt-iter (improve guess x)
                     x)))
```
What happens when Alyssa attempts to use this to compute square roots? Explain.

Using `new-if`, it takes long time without respon.

Becase `new-if` is a function. It must evaluate `sqrt-iter` in operand first, so it falls in to a infinite loop.But `if` is a special from, only if the `<predicate>` is `#f`,  `<alternative>` will be evaluted.

## Exercise 1.7
The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how `guess` changes from one iteration to the next and to stop when the change is a very small fraction of the guess. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers?

eg. For `(sqrt 0.0000000000001)` or `(sqrt 30864197530864197530864197530864197530864197530864197524691358024691358024691358024691358024691358024691358025)` the process dosen't work.
The improved program is below:

In [14]:
(define (improve guess x)
  (average guess (/ x guess)))
(define (average x y)
  (/ (+ x y) 2))
(define (good-enough? guess old-guess)
  (< (/ (abs (- old-guess guess)) 2) 0.001))
(define (sqrt x)
  (sqrt-iter 1.0 0.0 x))
(define (sqrt-iter guess old-guess x)
  (if (good-enough? guess old-guess)
          guess
          (sqrt-iter (improve guess x)
                     guess
                     x)))

## Exercise 1.8
Newton's method for cube roots is based on the fact that if y is an approximation to the cube root of x, then a better approximation is given by the value
$$\frac{x/y^2+2y}{3}$$
Use this formula to implement a cube-root procedure analogous to the square-root procedure. (In section 1.3.4 we will see how to implement Newton's method in general as an abstraction of these square-root and cube-root procedures.)

In [15]:
(define (improve guess x)
  (/ (+ (/ x (* guess guess)) (* 2 guess))
     3))
(define (good-enough? guess old-guess)
  (< (/ (abs (- old-guess guess)) 2) 0.001))
(define (cube-root x)
  (cube-root-iter 1.0 0.0 x))
(define (cube-root-iter guess old-guess x)
  (if (good-enough? guess old-guess)
          guess
          (cube-root-iter (improve guess x)
                     guess
                     x)))

## Exercise 1.9
Each of the following two procedures defines a method for adding two positive integers in terms of the procedures `inc`, which increments its argument by 1, and `dec`, which decrements its argument by 1.
```racket
(define (+ a b)
  (if (= a 0)
      b
      (inc (+ (dec a) b))))

(define (+ a b)
  (if (= a 0)
      b
      (+ (dec a) (inc b))))
```
Using the substitution model, illustrate the process generated by each procedure in evaluating `(+ 4 5)`. Are these processes iterative or recursive?

For the first process
```racket
(+ 4 5)
(inc (+ 3 5))
(inc (inc (+ 2 5)))
(inc (inc (inc (+ 1 5))))
(inc (inc (inc (inc (+ 0 5)))))
(inc (inc (inc (inc 5))))
(inc (inc (inc 6)))
(inc (inc 7))
(inc 8)
9
```
This is a recursive process.
```racket
(+ 4 5)
(+ 3 6)
(+ 2 7)
(+ 1 8)
(+ 0 9)
9
```
This is a iterative process.

## Exercise 1.10
The following procedure computes a mathematical function called Ackermann's function.
```racket
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1)
                 (A x (- y 1))))))
```
What are the values of the following expressions?
```racket
(A 1 10)
(A 2 4)
(A 3 3)
```
Consider the following procedures, where A is the procedure defined above:
```racket
(define (f n) (A 0 n))
(define (g n) (A 1 n))
(define (h n) (A 2 n))
(define (k n) (* 5 n n))
```
Give concise mathematical definitions for the functions computed by the procedures `f`, `g`, and `h` for positive integer values of $n$. For example, `(k n)` computes $5n^2$.


In [16]:
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1)
                 (A x (- y 1))))))

In [17]:
(A 1 10)

1024

In [18]:
(A 2 4)

65536

In [19]:
(A 3 3)

65536

`(f n)` computes $2n$

`(g n)` computes $2^n$

`(h n)` computes $2$ square $n - 1$ times and when $n = 0$, it's $0$

## Exercise 1.11
A function $f$ is defined by the rule that $f(n) = n$ if $n<3$ and $f(n) = f(n - 1) + 2f(n - 2) + 3f(n - 3)$ if $n> 3$. Write a procedure that computes $f$ by means of a recursive process. Write a procedure that computes $f$ by means of an iterative process.

In [20]:
; Recusive Process
(define (f n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        ((= n 2) 2)
        (else (+ (f (- n 1))
                 (* 2 (f (- n 2)))
                 (* 3 (f (- n 3)))))))

In [21]:
; Iterative Process
(define (f n)
  (define (iter a b c count)
    (if (= count 0)
        a
        (iter b c (+ (* 3 a) (* 2 b) c) (- count 1))))
  (iter 0 1 2 n))

## Exercise 1.12
The following pattern of numbers is called Pascal's triangle.

![h](./img/ch1-Z-G-17.gif)

The numbers at the edge of the triangle are all $1$, and each number inside the triangle is the sum of the two numbers above it. Write a procedure that computes elements of Pascal's triangle by means of a recursive process.

In [22]:
(define (element_of_Pascals_Triangle row column)
  (cond ((= column 1) 1)
        ((= row column) 1)
        (else (+ (element_of_Pascals_Triangle
                  (- row 1)
                  column)
                 (element_of_Pascals_Triangle
                  (- row 1)
                  (- column 1))))))

## Exercise 1.13
Prove that $Fib(n)$ is the closest integer to $\phi^n/\sqrt{5}$, where $\phi=(1+\sqrt{5})/2$. Hint: Let $\psi=(1-\sqrt{5})/2$. Use induction and definition of Fibonacci numbers. to prove that $Fib(n)= (\phi^n-\psi^n)/\sqrt{5}$.

If $k = 0$ and $k = 1$, 
$$Fib(0)=0 =(1-1)/\sqrt{5}$$
$$Fib(1) =1= ((1+\sqrt{5})/2-(1-\sqrt{5})/2)/\sqrt{5}$$
If $Fib(k-2)= (\phi^{k-2} -\psi^{k-2})/\sqrt{5}$ and  $Fib(k-1)= (\phi^{k-1}-\psi^{k-1})/\sqrt{5}$,

so I can prove that $Fib(k) = Fib(k-1)+ Fib(k-2)= (\phi^k-\psi^k)/\sqrt{5}$
$$err(k)=Fib(k)-\phi^n/
\sqrt{5}=-\psi^k/\sqrt{5}
$$
when $k=0$, $err(0)< 0.5$, and $err(k)$  is a monotonic descreasing functions and $err(k)> 0$, so $0<err(k)< 0.5$

So $Fib(n)$ is the closest integer to $\phi^n/\sqrt{5}$.

## Exercise 1.14
Draw the tree illustrating the process generated by the `count-change` procedure of section 1.2.2 in making change for 11 cents. What are the orders of growth of the space and number of steps used by this process as the amount to be changed increases?

![ans](./img/ans-ch1-14.png)
The branch that only change with \$1 coins takes the largest space, so the order of growth of space is $\Theta(n)$.

The branch that only change with \$1 coins, takes $\Theta(n)$ steps. Next, we go to the branch that change with \$5 and \$1 coins, every \$5 node has a $\Theta(n)$ subbranch that change with \$1. So the order of growth of steps of the \$5 \$1 branch is $\Theta(n^2)$. So on, the order of growth of steps of the branch \$3 c\$5 \$10 is $\Theta(n^3)$. So the order of growth of steps of is $\Theta(n^5)$

## Exercise 1.15
The sine of an angle (specified in radians) can be computed by making use of the approximation $\sin x \approx  x$ if $x$ is sufficiently small, and the trigonometric identity
$$\sin x = 3\sin \frac{x}{3}-4\sin^3{\frac{x}{3}}$$
to reduce the size of the argument of $\sin$. (For purposes of this exercise an angle is considered ``sufficiently small'' if its magnitude is not greater than 0.1 radians.) These ideas are incorporated in the following procedures:
```racket
(define (cube x) (* x x x))
(define (p x) (- (* 3 x) (* 4 (cube x))))
(define (sine angle)
   (if (not (> (abs angle) 0.1))
       angle
       (p (sine (/ angle 3.0)))))
```
a.  How many times is the procedure p applied when `(sine 12.15)` is evaluated?

b.  What is the order of growth in space and number of steps (as a function of `a`) used by the process generated by the `sine` procedure when `(sine a)` is evaluated?

a. `p` will be evaluate 5 times.

b. Order of growth in space and setps both are $\Theta (\log(a))$.

## Exercise 1.16
Design a procedure that evolves an iterative exponentiation process that uses successive squaring and uses a logarithmic number of steps, as does `fast-expt`. (Hint: Using the observation that $(b^{n/2})^2 = (b^2)^{n/2}$, keep, along with the exponent $n$ and the base $b$, an additional state variable $a$, and define the state transformation in such a way that the product a $b^n$ is unchanged from state to state. At the beginning of the process $a$ is taken to be $1$, and the answer is given by the value of $a$ at the end of the process. In general, the technique of defining an invariant quantity that remains unchanged from state to state is a powerful way to think about the design of iterative algorithms.)

In [23]:
(define (fast-expt b n)
  (fast-expt-iter b n 1))
(define (even? n)
  (= (remainder n 2) 0))
(define (fast-expt-iter b n a)
  (cond ((= n 0) a)
        ((even? n) (fast-expt-iter (* b b) (/ n 2) a))
        (else (fast-expt-iter b (- n 1) (* b a)))))

## Exercise 1.17
The exponentiation algorithms in this section are based on performing exponentiation by means of repeated multiplication. In a similar way, one can perform integer multiplication by means of repeated addition. The following multiplication procedure (in which it is assumed that our language can only add, not multiply) is analogous to the `expt` procedure:
```racket
(define (* a b)
  (if (= b 0)
      0
      (+ a (* a (- b 1)))))
```
This algorithm takes a number of steps that is linear in `b`. Now suppose we include, together with addition, operations `double`, which doubles an integer, and `halve`, which divides an (even) integer by 2. Using these, design a multiplication procedure analogous to `fast-expt` that uses a logarithmic number of steps.

In [24]:
(define (fast-mult b n)
  (cond ((= n 0) 0)
        ((even? n) (fast-mult (double b) (halve n)))
        (else (+ b (fast-mult b (- n 1))))))

## Exercise 1.18
Using the results of exercises 1.16 and 1.17, devise a procedure that generates an iterative process for multiplying two integers in terms of adding, doubling, and halving and uses a logarithmic number of steps.

In [25]:
(define (fast-mult b n)
  (fast-mult-iter b n 0))
(define (fast-mult-iter b n a)
  (cond ((= n 0) a)
        ((even? n) (fast-mult-iter (double b) (halve n) a))
        (else (fast-mult-iter b (- n 1) (+ a b)))))

## Exercise 1.19
There is a clever algorithm for computing the Fibonacci numbers in a logarithmic number of steps. Recall the transformation of the state variables `a` and `b` in the `fib-iter` process of section 1.2.2: $a \gets a + b$ and $b \gets a$. Call this transformation $T$, and observe that applying $T$ over and over again $n$ times, starting with $1$ and $0$, produces the pair $Fib(n+1)$ and $Fib(n)$. In other words, the Fibonacci numbers are produced by applying $T^n$, the nth power of the transformation $T$, starting with the pair $(1,0)$. Now consider $T$ to be the special case of $p = 0$ and $q = 1$ in a family of transformations $T_{pq}$, where $T_{pq}$ transforms the pair $(a,b)$ according to a $\gets bq + aq + ap$ and $b \gets bp + aq$. Show that if we apply such a transformation $T_{pq}$ twice, the effect is the same as using a single transformation $T_{p^{\prime}q^\prime}$ of the same form, and compute $p^\prime$ and $q^\prime$ in terms of $p$ and $q$. This gives us an explicit way to square these transformations, and thus we can compute $T^n$ using successive squaring, as in the `fast-expt` procedure. Put this all together to complete the following procedure, which runs in a logarithmic number of steps:
```racket
(define (fib n)
  (fib-iter 1 0 0 1 n))
(define (fib-iter a b p q count)
  (cond ((= count 0) b)
        ((even? count)
         (fib-iter a
                   b
                   <??>      ; compute p'
                   <??>      ; compute q'
                   (/ count 2)))
        (else (fib-iter (+ (* b q) (* a q) (* a p))
                        (+ (* b p) (* a q))
                        p
                        q
                        (- count 1)))))
```

$$a \gets bq + aq + ap$$
$$b \gets bp + aq$$
Transfrom twice:
$$a \gets (bp+aq)q + (bq+aq+ap)q + (bq+aq+ap) p$$
$$b \gets (bp+aq)p + (bq+aq+ap)aq$$
Simplify:
$$a \gets b(2pq+q^2) +a(2pq+q^2) + a(p^2+q^2)$$
$$b \gets b(p^2+q^2)+a(2pq+q^2)$$
So, transfrom twice can be understand as:
$$q\prime \gets 2pq+q^2$$
$$p\prime \gets p^2+q^2$$

In [26]:
(define (fib n)
  (fib-iter 1 0 0 1 n))
(define (fib-iter a b p q count)
  (cond ((= count 0) b)
        ((even? count)
         (fib-iter a
                   b
                   (+ (* q q) (* p p))          ; compute p'
                   (+ (* 2 p q) (* q q))        ; compute q'
                   (/ count 2)))
        (else (fib-iter (+ (* b q) (* a q) (* a p))
                        (+ (* b p) (* a q))
                        p
                        q
                        (- count 1)))))

TODO: next exercses!!!